In [1]:
!pip install pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
(train_images,train_labels), (test_images,test_labels) = keras.datasets.mnist.load_data()

print(f"trening -> {train_images.shape}")
print(f"trening -> {test_images.shape}")

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 0s 0us/step
trening -> (60000, 28, 28)
trening -> (10000, 28, 28)


In [4]:
train_images.shape

(1000, 784)

In [5]:
test_images.shape

(1000, 784)

In [6]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

In [7]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
#użycie wywołania zwrotnego w punkcie kontrolmym
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)


model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 1.2345 - sparse_categorical_accuracy: 0.6570
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 5s 55ms/step - loss: 1.2345 - sparse_categorical_accuracy: 0.6570 - val_loss: 0.7653 - val_sparse_categorical_accuracy: 0.7730
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.4686 - sparse_categorical_accuracy: 0.8570
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 27ms/step - loss: 0.4686 - sparse_categorical_accuracy: 0.8570 - val_loss: 0.5294 - val_sparse_categorical_accuracy: 0.8440
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.3165 - sparse_categorical_accuracy: 0.9120
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.3165 - sparse_categorical_accuracy: 0.9120 - val_loss: 0.4592 - val_sparse_categorical_accuracy: 0.8630
Epoch 4/10
29/32 [=======

In [9]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [10]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3571 - sparse_categorical_accuracy: 0.1000 - 201ms/epoch - 6ms/step
Untrained model, accuracy: 10.00%


In [11]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3950 - sparse_categorical_accuracy: 0.8790 - 97ms/epoch - 3ms/step
Restored model, accuracy: 87.90%
